# Initializing the SysML v2 API

In [12]:
from __future__ import print_function

import time
import requests
from pprint import pprint
import pandas as pd
import json

#host = "<specify protocol://host:port of the server that is a provider of the SysML v2 REST/HTTP API"
host = "Specify the URL of the SysML v2 API and Service Repository"

# Get projects

In [13]:
projects_url = f"{host}/projects" 
projects_response = requests.get(projects_url)

if projects_response.status_code == 200:
    projects = projects_response.json()
    
    df = pd.DataFrame({'Project Name':[], 'Project ID':[]})
    for p in projects:
        df = pd.concat([df, pd.DataFrame({'Project Name':[p['name']], 'Project ID':[p['@id']]})], ignore_index=True)
    display(df)

,Project Name,Project ID
0,5-State-based Behavior-1a Fri Mar 10 17:37:14 ...,024b1aa6-7489-4106-bf2d-498f441f3d56
1,13a-Model Containment Fri Mar 10 16:52:40 EST ...,083a7c88-757c-41ff-89db-bc34a12b7794
2,SimpleQuadcopter Fri Mar 10 17:54:58 EST 2023,092dd68d-887f-4553-8ed2-05d405fb3a47
3,15_05-Unification of Expression and Constraint...,098d5715-5dee-46d1-bdf8-ab96f1cc9fc6
4,7b-Variant Configurations Fri Mar 10 17:38:53 ...,0b773e83-226e-4e20-ad89-5cbf1381b401
...,...,...
89,14b-Language Extensions Fri Mar 10 17:49:37 ES...,f97580aa-646c-4895-a8cb-006cd7538bb6
90,18-Use Case Fri Mar 10 17:00:37 EST 2023,fa2bae57-4b5e-46e0-a0ef-ede2f121ea1f
91,AnalysisAnnotation2 Wed Aug 30 04:36:12 UTC 2023,fb6d4f79-04e0-4a73-b11a-09263b79d7c3
92,11b-Safety and Security Feature Views Fri Mar ...,fc58c0ac-58ea-4a19-90d7-c9e2f1c6da84


# Define functions to get owned elements - immediate and recursive
## Define a function to get an element and print its name and type

In [14]:
def get_element(project_id, commit_id, element_id, indent):
     # Fetch the element in the given commit of the given project
    element_url = f"{host}/projects/{project_id}/commits/{commit_id}/elements/{element_id}" 
    response = requests.get(element_url)
    
    if response.status_code == 200:
        element_data = response.json()
        element_name_to_print = element_data['name'] if element_data['name'] else 'N/A'
        element_type = element_data ['@type']
        print(f"{indent} - {element_name_to_print} ({element_type})")
        return element_data
    else:
        return None

## Define a function to get owned elements (immediate) for a given element

In [15]:
# Returns direct / immediate owned elements for a given element in a given commit of a given project
def get_owned_elements_immediate(project_id, commit_id, element_id, indent):
    
    # Fetch the element in the given commit of the given project
    element_data = get_element(project_id, commit_id, element_id, indent)
    
    if element_data:
        owned_elements = element_data['ownedElement']
        if len(owned_elements) > 0:
            for owned_element in owned_elements:
                get_element(project_id, commit_id, owned_element['@id'], indent + '  ')
    else:
        print(f"Unable to fetch element with id '{element_id}' in commit '{commit_id}' of project '{project_id}'")

## Define a function to get owned elements (recursive) for a given element

In [16]:
# Returns directly / immediate owned elements for a given element in a given commit of a given project
def get_owned_elements(project_id, commit_id, element_id, indent):
    
    # Fetch the element in the given commit of the given project
    element_data = get_element(project_id, commit_id, element_id, indent)
    
    if element_data:
        owned_elements = element_data['ownedElement']
        if len(owned_elements) > 0:
            for owned_element in owned_elements:
                get_owned_elements(project_id, commit_id, owned_element['@id'], indent+' ')
    else:
        print(f"Unable to fetch element with id '{element_id}' in commit '{commit_id}' of project '{project_id}'")

# Test for an example element in a given commit of a project

## Specify test project, commit, and element

In [17]:
# Specify a project id
test_project_id = 'b0c4aec0-8fce-4d82-af3d-f9abdd25af19' # 1c-Parts Tree Redifinition project

# Specify a commit id in the project
test_commit_id = 'caa160a2-e964-4545-bb3c-c7686d5d2979' # Latest commit in the main branch of the project

# Specify an element id in the commit of that project
test_element_id = '2a361c0a-a332-4910-a288-c8b4b35bd2ab' #1c-Parts Tree Redifinition package

## Get owned elements (immediate) for the given element in the given commit of the given project

In [18]:
get_owned_elements_immediate(test_project_id, test_commit_id, test_element_id, '')

 - 1c-Parts Tree Redefinition (Package)
   - Definitions (Package)
   - Usages (Package)


## Get owned elements (recursive) for the given element in the given commit of the given project

In [19]:
get_owned_elements(test_project_id, test_commit_id, test_element_id, '')

 - 1c-Parts Tree Redefinition (Package)
  - Definitions (Package)
   - Vehicle (PartDefinition)
    - mass (AttributeUsage)
     - N/A (Multiplicity)
   - AxleAssembly (PartDefinition)
   - Axle (PartDefinition)
    - mass (AttributeUsage)
     - N/A (Multiplicity)
   - FrontAxle (PartDefinition)
    - steeringAngle (AttributeUsage)
     - N/A (Multiplicity)
   - Wheel (PartDefinition)
  - Usages (Package)
   - vehicle1 (PartUsage)
    - mass (AttributeUsage)
     - N/A (OperatorExpression)
      - x (Feature)
       - N/A (LiteralInteger)
        - result (Feature)
      - y (Feature)
       - N/A (FeatureReferenceExpression)
        - result (Feature)
      - result (Feature)
     - N/A (Documentation)
    - frontAxleAssembly (PartUsage)
     - frontAxle (PartUsage)
      - N/A (Multiplicity)
     - frontWheel (PartUsage)
      - N/A (MultiplicityRange)
       - N/A (LiteralInteger)
        - result (Feature)
     - N/A (Multiplicity)
    - rearAxleAssembly (PartUsage)
     - rearAxl